In [42]:
import numpy as np

# FUNCTIONS

In [3]:
def f1(x, y, z):
    return (x - z) ** 2 + (2*y + z) ** 2 + (4*x -2*y + z) ** 2 + x + y

def f2(x, y, z):
    return ((x - 1) ** 2) + ((y - 1) ** 2) + 100*((y - (x ** 2)) ** 2) + 100*((z - (y ** 2)) ** 2)

def f3(x, y):
    return (1.5 - x + x*y) ** 2 + (2.25 - x + x*(y ** 2)) ** 2 + (2.625 - x + x*(y ** 3)) ** 2

# 2D NELDER MEAD

I will first implement only the 2 dimensional version in order to try to figure out the core logics.

In [ ]:
def nelder_mead_2d(v1, v2, v3, f):

    points = [v1, v2, v3]

    #the loop should start here
    for i in range(5):
        zs = []
        
        for point in points:
            zs.append(f(*point))

        sorted_indices = np.argsort(zs)    

        best = points[sorted_indices[0]]
        good = points[sorted_indices[1]]
        worst = points[sorted_indices[2]]

        #Sanity check
        # print(best, f(*best))
        # print(good, f(*good))
        # print(worst, f(*worst))

        M = (best + good) / 2
        R = 2*M - worst
        E = 2*R - M
        # print(M, R, E)

        yr = f(*R)
        ye = f(*E)

        zs.sort() #sort them so i can easily compare the values

        if yr < zs[0]:
            if ye < yr:
                next = E
            else:
                next = R
            points = [best, good, next]
        
        elif yr < zs[1]:
            next = R
            points = [best, good, next]
        
        elif yr < zs[2]:
            oc = worst + 3/2 * (M - worst)
            yoc = f(*oc)
            if yoc < yr:
                next = oc
            points = [best, good, next]
        else: #shrink
            good = 1/2 * (best + good)
            worst = 1/2 * (best + worst)
            points = [best, good, worst]

        print(f(*best))



In [66]:
test_point1 = np.array([0, 0])
test_point2 = np.array([1, 0])
test_point3 = np.array([3, 2])

nelder_mead_2d(test_point1, test_point2, test_point3, f3)


[1 0] 4.453125
[0 0] 14.203125
[3 2] 704.953125
4.453125
[1 0] 4.453125
[0 0] 14.203125
[-0.75 -1.  ] 31.078125
4.453125
[1 0] 4.453125
[1.125 0.5  ] 5.548095703125
[0 0] 14.203125
4.453125
[2.125 0.5  ] 1.208251953125
[1 0] 4.453125
[1.125 0.5  ] 5.548095703125
1.208251953125
[2. 0.] 0.703125
[2.125 0.5  ] 1.208251953125
[1 0] 4.453125
0.703125
